In [1]:
import pandas as pd
import numpy as np


# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Realizar los distintos test
# ------------------------------------------------------------------------------

from scipy.stats import ttest_ind, norm, chi2_contingency, f_oneway
from sklearn.linear_model import LinearRegression


# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

C:\Users\Isabel\AppData\Local\Temp\ipykernel_13844\4252833058.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_csv('marketing_AB.csv', index_col=0)

df.head

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14


In [12]:
df['test group'].unique()

array(['ad', 'psa'], dtype=object)

In [9]:
df.shape

(588101, 6)

In [5]:
df.isnull().sum()

user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64

In [6]:
df.describe()

,user id,total ads,most ads hour
count,5.881010e+05,588101.000000,588101.000000
mean,1.310692e+06,24.820876,14.469061
std,2.022260e+05,43.715181,4.834634
min,9.000000e+05,1.000000,0.000000
25%,1.143190e+06,4.000000,11.000000
50%,1.313725e+06,13.000000,14.000000
75%,1.484088e+06,27.000000,18.000000
max,1.654483e+06,2065.000000,23.000000


In [7]:
df.describe(include='object')

,test group,most ads day
count,588101,588101
unique,2,7
top,ad,Friday
freq,564577,92608


In [8]:
df.duplicated().sum()

0

In [11]:
for columna in df.columns:
    print(f'Columna: {columna}. Duplicados: {df[columna].duplicated().sum()}')

Columna: user id. Duplicados: 0
Columna: test group. Duplicados: 588099
Columna: converted. Duplicados: 588099
Columna: total ads. Duplicados: 587294
Columna: most ads day. Duplicados: 588094
Columna: most ads hour. Duplicados: 588077


# Hipótesis

H0: No hay diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".

H1: Existe una diferencia significativa en la tasa de conversión entre los grupos de prueba "ad" y "psa".

In [16]:
df.sample(5)

,user id,test group,converted,total ads,most ads day,most ads hour
576925,1194990,ad,False,2,Sunday,12
369045,1077776,ad,False,1,Sunday,21
272318,1339873,ad,False,33,Wednesday,13
331442,1509061,ad,False,3,Thursday,12
263232,1548307,ad,False,2,Tuesday,10


In [14]:
df_ad= df[df['test group']== 'ad']
tasa_ad_converted = np.round(df_ad[df_ad['converted'] == True].shape[0] / df_ad.shape[0],3)

df_psa = df[df['test group'] == 'psa']
tasa_psa_converted = np.round(df_psa[df_psa['converted'] == True].shape[0]/df_psa.shape[0],3)

print(f'Tasa conversión nuevo diseño: {tasa_ad_converted}')
print(f'Tasa conversión antiguo diseño: {tasa_psa_converted}')

Tasa conversión nuevo diseño: 0.026
Tasa conversión antiguo diseño: 0.018


In [17]:
from scipy.stats import chi2_contingency

In [19]:
df2 =pd.DataFrame({'Grupo': ['ad', 'ps'],'tasa_converted': [np.round(df_ad[df_ad['converted'] == True].shape[0] / df_ad.shape[0],3),np.round(df_psa[df_psa['converted'] == True].shape[0]/df_psa.shape[0],3) ],
         'tasa_sin_converted' : [np.round(df_ad[df_ad['converted'] == False].shape[0] / df_ad.shape[0],3),np.round(df_psa[df_psa['converted'] == False].shape[0]/df_psa.shape[0],3) ]})

In [20]:
df2

,Grupo,tasa_converted,tasa_sin_converted
0,ad,0.026,0.974
1,ps,0.018,0.982


In [21]:
tabla_contingencia = pd.crosstab(df2['Grupo'], df2['tasa_converted'])
chi2, p_value, _, _ = chi2_contingency(tabla_contingencia)

alpha = 0.05
if p_value < alpha:
   print("Hay una diferencia significativa en las tasas de conversiíon entre ambos grupos")
else:
  print("No hay evidencia de una diferencia significativa en las tasas de conversión entre los grupos.")

No hay evidencia de una diferencia significativa en las tasas de conversión entre los grupos.


In [24]:
t_stat, p_value = ttest_ind(tasa_ad_converted, tasa_psa_converted)

if p_value < 0.05:
    print("Hay una diferencia significativa en las tasas de clics entre el grupo de control y el grupo de prueba.")
    print("\n ---------- \n")
    print("""
          Los resultados sugieren que existe evidencia estadística para afirmar que las medias de las muestras son distintas. 
          Por lo tanto, nuestro nuevo sistema tiene los efectos deseados y deberíamos cambiar la nueva versión de anuncios   
          """)
else:
    print("No hay evidencia de una diferencia significativa en las tasas de clics entre los grupos.")
    print("\n ---------- \n")
    print("""
          Los resultados sugieren que no existe evidencia estadística para afirmar que las medias de las muestras son distintas. 
          Por lo tanto, aunque en la gráfica veamos que hay diferencias entre el grupo "test" y el grupo "control", 
          el análisis estadístico nos sugiera que estas diferencias no son significativas, 
          por lo que la nueva campaña no esta ayudando a nuestro problema.   
          """)

No hay evidencia de una diferencia significativa en las tasas de clics entre los grupos.

 ---------- 


          Los resultados sugieren que no existe evidencia estadística para afirmar que las medias de las muestras son distintas. 
          Por lo tanto, aunque en la gráfica veamos que hay diferencias entre el grupo "test" y el grupo "control", 
          el análisis estadístico nos sugiera que estas diferencias no son significativas, 
          por lo que la nueva campaña no esta ayudando a nuestro problema.   
          


# Ejercicio 2

- H0: los anuncios no suponen una diferencia significativa
- H1: los anuncios suponen una diferencia significativa

In [25]:
df.columns

Index(['user id', 'test group', 'converted', 'total ads', 'most ads day',
       'most ads hour'],
      dtype='object')

In [32]:
df3 = pd.DataFrame({'grupo': ['ad', 'psa'], 
                    'anuncios vistos': [
                        df[df['test group']=='ad']['total ads'].mean(), 
                        df[df['test group']=='psa']['total ads'].mean()]})

In [33]:
df3

,grupo,anuncios vistos
0,ad,24.823365
1,psa,24.761138


In [34]:
t_stat, p_value = ttest_ind(df3.iloc[0,1], df3.iloc[1,1])

if p_value < 0.05:
    print("Hay una diferencia significativa en las tasas de clics entre el grupo de control y el grupo de prueba.")
    print("\n ---------- \n")
    print("""
          Los resultados sugieren que existe evidencia estadística para afirmar que las medias de las muestras son distintas. 
          Por lo tanto, nuestro nuevo sistema tiene los efectos deseados y deberíamos cambiar la nueva versión de anuncios   
          """)
else:
    print("No hay evidencia de una diferencia significativa en las tasas de clics entre los grupos.")
    print("\n ---------- \n")
    print("""
          Los resultados sugieren que no existe evidencia estadística para afirmar que las medias de las muestras son distintas. 
          Por lo tanto, aunque en la gráfica veamos que hay diferencias entre el grupo "test" y el grupo "control", 
          el análisis estadístico nos sugiera que estas diferencias no son significativas, 
          por lo que la nueva campaña no esta ayudando a nuestro problema.   
          """)

No hay evidencia de una diferencia significativa en las tasas de clics entre los grupos.

 ---------- 


          Los resultados sugieren que no existe evidencia estadística para afirmar que las medias de las muestras son distintas. 
          Por lo tanto, aunque en la gráfica veamos que hay diferencias entre el grupo "test" y el grupo "control", 
          el análisis estadístico nos sugiera que estas diferencias no son significativas, 
          por lo que la nueva campaña no esta ayudando a nuestro problema.   
          
